In [1]:
print("hello")

hello


In [49]:
%pwd

'C:\\Users\\Toshiba\\Desktop\\Medical-Chatbot-GenAI\\Medical-Chatbot-GenAI\\research'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [28]:
#Extract Data From the PDF

def load_pdf_file(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [51]:
extracted_data = load_pdf_file(data="../Data/")


In [52]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [53]:
text_chunks=text_split(extracted_data)
print("Lenght of Text Chunks", len(text_chunks))

Lenght of Text Chunks 6970


In [54]:
from langchain.embeddings import HuggingFaceEmbeddings

In [55]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [60]:
embeddings = download_hugging_face_embeddings()

In [61]:
query_results = embeddings.embed_query("hello world")
print("lenght", len(query_results))

lenght 384


In [104]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [105]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [87]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key= PINECONE_API_KEY)

index_name = "medaibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [106]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [89]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [90]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [91]:
docsearch

In [92]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [93]:
retrieved_docs = retriever.invoke("what is Contraception?")

In [94]:
retrieved_docs

[Document(id='2d6373c2-0f98-4733-a17a-9b29cfd5be32', metadata={'page': 298.0, 'page_label': '299', 'source': '..\\Data\\Medical_Book.pdf'}, page_content='therapies\nContraception\nDefinition\nContraception (birth control) prevents pregnancy\nby interfering with the normal process of ovulation, fer-\nGALE ENCYCLOPEDIA OF MEDICINE 2910\nContraception'),
 Document(id='d9e91dac-602a-480a-b417-535068b16614', metadata={'page': 299.0, 'page_label': '300', 'source': '..\\Data\\Medical_Book.pdf'}, page_content='egg. Any woman who wants to prevent pregnancy must\nuse a reliable form of birth control.\nBirth control (contraception) is designed to interfere\nwith the normal process and prevent the pregnancy that\ncould result. There are different kinds of birth control\nthat act at different points in the process, from ovulation,\nthrough fertilization, to implantation. Each method has\nits own side effects and risks. Some methods are more\nreliable than others.\nAlthough there are many different 

In [98]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [99]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you dont't know the answer, say that you "
    "dont't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [100]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [103]:
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=2, min=8, max=20))
def query_model(input_text):
    return rag_chain.invoke({"input": input_text})

try:
    response = query_model("What is Contraception?")
    print(response["answer"])
except Exception as e:
    print("Error:", e)

Error: RetryError[<Future at 0x2242ed06ef0 state=finished raised RateLimitError>]
